In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm

def 파일명(폴더:str, 키워드:str) :
    file_list = os.listdir(폴더)
    account_filename = [file for file in file_list if re.search(키워드, file)]
    if len(account_filename) == 1 :
        return join(폴더,account_filename[0])
    else :
        print("해당 키워드가 들어간 파일이 하나가 아닙니다.")

In [13]:
# 채무자 파일읽기
####################################################
폴더 = r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\이름"
kw_df = "대성_보증인.xlsx"
####################################################     
df = pd.read_excel(join(폴더, kw_df), dtype={"채무자키" : str,"보증인키":str})[["보증인키", "채무자키", "보증인성명", "메모"]]
# df = pd.read_pickle(파일명(폴더, kw_df))
df

,보증인키,채무자키,보증인성명,메모
0,1042182,10622651,공병국,NaN
1,1042183,10623732,안현주,NaN
2,1042184,10623794,복남규,NaN
3,1042185,10623795,복남규,NaN
4,1042186,10623795,조무학,NaN
...,...,...,...,...
405,1054926,10925611,박성혜,NaN
406,1054927,10925612,이지용,NaN
407,1054928,10925613,박병수,NaN
408,1054929,10925614,박병수,NaN


In [14]:
# 법조치 최신화를 위해 처리할 것
# 1. (유),(주),(.) : 나우리에서 자동삭제되므로 pass, 공백은 법원에서 자동삭제되므로 pass
# 2. 괄호제외한 기타 기호는 삭제되지 않으므로 제거해야함

pattern = re.compile("[^a-zA-Z가-힣\s㈜]")
기호포함건 = df[df["보증인성명"].str.contains(pattern, regex=True)]
기호포함건

,보증인키,채무자키,보증인성명,메모


In [ ]:
기호포함건.to_excel("xxx.xlsx", index=False)

In [79]:
# # 종결건 제외하기 위해 전체 원데이터 읽기
kw_raw = r"2.솔림_전체_원데이터_240108_by새채무자키_제외계좌"
raw_data = pd.read_pickle(파일명(폴더, kw_raw))

# 종료 아닌 채무자키
관리채무자키 = raw_data.query('~새채무상태.str.contains("종료")').채무자키

In [84]:
##########################################
cols = ["채무자키","분납키","분납자관계" ,"분납자성명인"] #["채무자키","보증인키", "보증인성명인"] # 
성명칼럼명 = cols[-1] # 성명 부분
##########################################
작업파일 = df[df.채무자키.isin(관리채무자키)][cols]

In [85]:
# 법조치 최신화를 위해 처리할 것
# 1. (유),(주),(.) : 나우리에서 자동삭제되므로 pass, 공백은 법원에서 자동삭제되므로 pass
# 2. 괄호제외한 기타 기호는 삭제되지 않으므로 제거해야함

pattern = re.compile("[^a-zA-Z가-힣\s()㈜]")
기호포함건 = 작업파일[작업파일[성명칼럼명].str.contains(pattern, regex=True)]

pattern1 = re.compile(r"([a-zA-Z가-힣\d]+)")
기호포함건["새성명"] = 기호포함건[성명칼럼명].str.extract(pattern1)
기호포함건

C:\Users\DATA\AppData\Local\Temp\ipykernel_12900\812053915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  기호포함건["새성명"] = 기호포함건[성명칼럼명].str.extract(pattern1)


,채무자키,분납키,분납자관계,분납자성명인,새성명
49,20408458,20106012,채무자,채원종(변경전 :채원종(변경전,채원종
77,20408535,20107221,채무자,배채문/창원건설기계,배채문
78,20408538,20104970,채무자,주영근/대길통운,주영근
79,20408539,20106905,채무자,신동준/두리통신,신동준
80,20408540,20106013,채무자,현신우/태양사랑운수,현신우
81,20408541,20104832,채무자,지호선/로지스퀘어,지호선
82,20408542,20107199,채무자,김성희/화인토건,김성희
83,20408544,20107198,채무자,박경남/경남식품,박경남
84,20408545,20104593,채무자,김인곤/GS25고양원당,김인곤
85,20408548,20106103,채무자,김장한/장한용달화물,김장한


In [82]:
기호포함건.to_excel("xxx.xlsx", index=False)